<a href="https://colab.research.google.com/github/opinsky/learnbezier/blob/master/learning_try3_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# this program has the following structure:
# input = 100 pairs of x-y points
# output = 8 pairs of x-y that are the control points of a bezier curve
!pip install geomdl

     |████████████████████████████████| 143kB 4.1MB/s 


In [0]:
%matplotlib inline
from geomdl import BSpline
from geomdl import utilities

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import random
import numpy as np
tf.enable_eager_execution()

In [0]:
# those are utilities to generate the training data
n_evalpts = 100
def evalpts(ctrlpts):
    c = BSpline.Curve()
    c.degree = 2
    c.ctrlpts = ctrlpts
    c.delta = 1/(n_evalpts-0.5)
    c.knotvector = utilities.generate_knot_vector(c.degree, len(c.ctrlpts))
    c.render
    return c.evalpts

def mevalpts(ctrlptslist):
    evalptslist=[]
    for i in range(len(ctrlptslist)):
        evalptslist.append(evalpts(ctrlptslist[i]))
    return evalptslist
 
#curve = evalpts([[5.0, 10.0], [15.0, 25.0], [30.0, 30.0], [45.0, 5.0], [55.0, 5.0],
#                 [70.0, 40.0], [60.0, 60.0], [35.0, 60.0]])

In [4]:
#generate X samples of inputs (100 pairs of points) and outputs (8 pairs)
NUM_SAMPLES = 30000
outputs = tf.random_uniform(shape=[NUM_SAMPLES, 8,2], maxval=5)
inputs = np.array(mevalpts(outputs.numpy().tolist())) #shape [n_evalpts,2]
#inputs = tf.Variable(mevalpts(outputs.numpy().tolist()))
print(inputs.shape)
print(outputs.shape)

(30000, 100, 2)
(30000, 8, 2)


In [0]:
def model():
    model = keras.Sequential([
        layers.Flatten(input_shape=(n_evalpts,2)),
        layers.Dense(256, activation=tf.nn.relu),
        layers.Dense(128, activation=tf.nn.relu),
        layers.Dense(64, activation=tf.nn.relu),
        layers.Dense(32, activation=tf.nn.relu),
        layers.Dense(16)
    ])
    #optimizer = tf.keras.optimizers.RMSprop(0.001)
    optimizer = tf.keras.optimizers.Adam()
    model.compile(
                  loss="mean_squared_error",
                  #loss=loss,  #'mean_squared_error',
                  optimizer=optimizer,
                  metrics=['mean_absolute_error', 'mean_squared_error'])
    return model

In [0]:
def loss(predicted_evalpts, desired_evalpts):
    return tf.reduce_mean(tf.square(predicted_evalpts - desired_evalpts))

In [7]:
model = model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               51456     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
Total params: 95,216
T

In [8]:
loss(outputs,np.reshape(model.predict(np.array(inputs)),(NUM_SAMPLES,8,2)))

<tf.Tensor: id=3194, shape=(), dtype=float32, numpy=10.586954>

In [0]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 50 == 0:
            print ("loss: %.6f, val_loss: %.6f" % (logs["loss"], logs["val_loss"]))
            print('')
        print('.', end='')
    #def on_train_end(self, logs):
    #    print (logs["loss"])

EPOCHS = 5000
history = model.fit(
    inputs,
    np.reshape(outputs,(NUM_SAMPLES,16)),
    epochs=EPOCHS,
    validation_split = 0.05,
    verbose=0,
    callbacks=[PrintDot()]
  )

Instructions for updating:
Use tf.cast instead.
loss: 0.854517, val_loss: 0.484542

..................................................loss: 0.254993, val_loss: 0.266337

..................................................loss: 0.253879, val_loss: 0.262205

..................................................loss: 0.253476, val_loss: 0.265879

..................................................loss: 0.253143, val_loss: 0.263270

..................................................loss: 0.253042, val_loss: 0.268867

..................................................loss: 0.252892, val_loss: 0.263753

..................................................loss: 0.252533, val_loss: 0.264874

..................................................loss: 0.252496, val_loss: 0.266424

..................................................loss: 0.252252, val_loss: 0.263803

..................................................loss: 0.252301, val_loss: 0.263531

..................................................loss: 

In [0]:
from matplotlib import pyplot as plt
w = tf.random_uniform(shape=[8,2], maxval=15)
i = evalpts(np.asarray(w).tolist())
x, y = np.asarray(i).T
o = evalpts(np.reshape(model.predict(np.reshape(np.asarray(i),(1,n_evalpts,2))),(8,2)).tolist())
a, b = np.asarray(o).T
plt.scatter(x,y, c="b")
plt.scatter(a,b, c="r")

In [0]:
#model.save_weights("./beziermodelweights")